# Advanced Chatbot using LangGraph with Groq + RAG

This notebook demonstrates a sophisticated chatbot that supports:
- Normal conversations using Groq's fast LLM
- RAG (Retrieval-Augmented Generation) for accurate information retrieval
- Document upload functionality for external documents
- Memory and persistence to maintain context across sessions
- LangSmith integration for tracing and debugging
- User-friendly interface

## Setup and Dependencies

In [ ]:
# Install required packages if not already installed
!pip install langgraph langchain langchain-groq langsmith chromadb sentence-transformers pypdf pymupdf python-docx

In [1]:
import os
import tempfile
from typing import Dict, List, Any, TypedDict, Annotated
from datetime import datetime
import json
import glob

from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.document_loaders import (
    PyPDFLoader, 
    TextLoader, 
    Docx2txtLoader,
    UnstructuredFileLoader
)

from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver

import langsmith

print("All imports successful!")

All imports successful!


In [2]:
# Set up environment variables
# You'll need to set these in your environment or use a .env file
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "advanced-chatbot-groq-rag"

# Set your Groq API key
# os.environ["GROQ_API_KEY"] = "your-groq-api-key-here"

# Optional: Set your LangSmith API key for tracing
# os.environ["LANGCHAIN_API_KEY"] = "your-langsmith-api-key-here"

print("Environment configured!")

Environment configured!


In [3]:
class ChatState(TypedDict):
    """State for the chatbot conversation"""
    messages: Annotated[List[Any], "The conversation messages"]
    user_id: Annotated[str, "The user ID for session management"]
    context: Annotated[Dict[str, Any], "Additional context for the conversation"]
    retrieved_docs: Annotated[List[str], "Retrieved documents for RAG"]
    current_step: Annotated[str, "Current processing step"]

print("State structure defined!")

State structure defined!


In [4]:
# Initialize Groq language model
# You can choose from different Groq models:
# - "llama3-8b-8192" (fast, good for chat)
# - "llama3-70b-8192" (slower but more capable)
# - "mixtral-8x7b-32768" (good balance)
# - "gemma2-9b-it" (fast and efficient)

llm = ChatGroq(
    model="llama3-8b-8192",  # Fast and good for chat
    temperature=0.7,
    groq_api_key=os.getenv("GROQ_API_KEY")
)

print("Groq model initialized!")

Groq model initialized!


In [5]:
def load_document(file_path: str):
    """Load document based on file extension"""
    file_extension = file_path.lower().split('.')[-1]
    
    try:
        if file_extension == 'pdf':
            loader = PyPDFLoader(file_path)
        elif file_extension == 'txt':
            loader = TextLoader(file_path)
        elif file_extension == 'docx':
            loader = Docx2txtLoader(file_path)
        else:
            # Try unstructured loader for other file types
            loader = UnstructuredFileLoader(file_path)
        
        documents = loader.load()
        print(f"Successfully loaded {len(documents)} pages/sections from {file_path}")
        return documents
        
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return []

def load_documents_from_directory(directory_path: str):
    """Load all supported documents from a directory"""
    all_documents = []
    
    # Supported file extensions
    supported_extensions = ['*.pdf', '*.txt', '*.docx', '*.md']
    
    for extension in supported_extensions:
        file_pattern = os.path.join(directory_path, extension)
        files = glob.glob(file_pattern)
        
        for file_path in files:
            documents = load_document(file_path)
            all_documents.extend(documents)
    
    print(f"Total documents loaded: {len(all_documents)}")
    return all_documents

print("Document loading functions defined!")

Document loading functions defined!


In [6]:
# Sample knowledge base (you can replace this with your own documents)
sample_docs = [
    "LangGraph is a library for building stateful, multi-actor applications with LLMs.",
    "RAG (Retrieval-Augmented Generation) combines retrieval with text generation for more accurate responses.",
    "LangSmith is a platform for debugging, testing, evaluating, and monitoring LLM applications.",
    "Memory in chatbots helps maintain context across conversation turns.",
    "Vector databases store embeddings for efficient similarity search.",
    "LangChain provides building blocks for LLM applications.",
    "Groq is a fast LLM inference platform that provides ultra-low latency responses.",
    "Groq's models include Llama, Mixtral, and Gemma variants optimized for speed.",
    "Python is a high-level programming language known for its simplicity and readability.",
    "Machine learning is a subset of artificial intelligence that enables computers to learn from data.",
    "Natural language processing (NLP) helps computers understand and generate human language.",
    "Deep learning uses neural networks with multiple layers to solve complex problems.",
    "Data science combines statistics, programming, and domain expertise to extract insights from data.",
    "Cloud computing provides on-demand access to computing resources over the internet.",
    "APIs (Application Programming Interfaces) allow different software systems to communicate with each other."
]

print("Sample knowledge base created!")

Sample knowledge base created!


In [8]:
# Create embeddings and vector store
try:
    # Use a simple, reliable embedding model
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2",
        model_kwargs={'device': 'cpu'}  # Use CPU to avoid GPU issues
    )
    
    # Split documents into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=500, 
        chunk_overlap=50
    )
    
    # Create documents from sample text
    sample_documents = text_splitter.create_documents(sample_docs)
    
    # Create vector store
    vectorstore = Chroma.from_documents(
        sample_documents, 
        embeddings,
        collection_name="chatbot_knowledge"
    )
    
    # Create retriever
    retriever = vectorstore.as_retriever(
        search_type="similarity",
        search_kwargs={"k": 3}
    )
    
    print("Vector store and retriever created successfully!")
    print(f"Initial knowledge base contains {len(sample_documents)} chunks")
    
except Exception as e:
    print(f"Warning: Could not create vector store: {e}")
    print("The chatbot will work without RAG functionality.")
    retriever = None
    vectorstore = None

Vector store and retriever created successfully!
Initial knowledge base contains 15 chunks


In [ ]:
def add_documents_to_knowledge_base(file_paths: List[str]):
    """Add uploaded documents to the knowledge base"""
    global vectorstore, retriever
    
    if vectorstore is None:
        print("Vector store not initialized. Cannot add documents.")
        return False
    
    try:
        all_documents = []
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500, 
            chunk_overlap=50
        )
        
        # Load documents from files
        for file_path in file_paths:
            if os.path.isfile(file_path):
                documents = load_document(file_path)
                all_documents.extend(documents)
            elif os.path.isdir(file_path):
                documents = load_documents_from_directory(file_path)
                all_documents.extend(documents)
        
        if not all_documents:
            print("No documents were loaded successfully.")
            return False
        
        # Split documents into chunks
        split_documents = text_splitter.split_documents(all_documents)
        
        # Add to vector store
        vectorstore.add_documents(split_documents)
        
        # Update retriever
        retriever = vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": 3}
        )
        
        print(f"Successfully added {len(split_documents)} document chunks to knowledge base")
        print(f"Total knowledge base size: {vectorstore._collection.count()} chunks")
        
        return True
        
    except Exception as e:
        print(f"Error adding documents to knowledge base: {e}")
        return False

def list_knowledge_base_info():
    """List information about the current knowledge base"""
    if vectorstore is None:
        print("No vector store available.")
        return
    
    try:
        count = vectorstore._collection.count()
        print(f"Knowledge base contains {count} document chunks")
        
        # Get some sample documents
        sample_docs = vectorstore.similarity_search("", k=5)
        print(f"Sample documents in knowledge base:")
        for i, doc in enumerate(sample_docs[:3], 1):
            print(f"{i}. {doc.page_content[:100]}...")
            
    except Exception as e:
        print(f"Error getting knowledge base info: {e}")

print("Document upload functions defined!")

In [ ]:
def should_use_rag(state: ChatState) -> str:
    """Determine if we should use RAG based on the user's question"""
    if retriever is None:
        return "chat"  # Fallback to chat if no retriever
    
    last_message = state["messages"][-1].content.lower()
    
    # Keywords that suggest the user wants factual information
    rag_keywords = [
        "what is", "how does", "explain", "tell me about", 
        "information", "knowledge", "define", "describe",
        "what are", "how to", "guide", "tutorial", "find",
        "search", "look for", "what does", "where is"
    ]
    
    # Check if the message contains RAG keywords
    if any(keyword in last_message for keyword in rag_keywords):
        return "rag"
    else:
        return "chat"

print("RAG decision function defined!")

In [ ]:
def retrieve_context(state: ChatState) -> ChatState:
    """Retrieve relevant documents for RAG"""
    if retriever is None:
        state["retrieved_docs"] = []
        state["current_step"] = "no_retriever"
        return state
    
    try:
        query = state["messages"][-1].content
        docs = retriever.get_relevant_documents(query)
        
        state["retrieved_docs"] = [doc.page_content for doc in docs]
        state["current_step"] = "retrieved_context"
        
        print(f"Retrieved {len(state['retrieved_docs'])} relevant documents")
        
    except Exception as e:
        print(f"Error retrieving documents: {e}")
        state["retrieved_docs"] = []
        state["current_step"] = "retrieval_error"
    
    return state

print("Context retrieval function defined!")

In [ ]:
def generate_rag_response(state: ChatState) -> ChatState:
    """Generate response using RAG"""
    if not state["retrieved_docs"]:
        # Fallback to chat if no documents retrieved
        return generate_chat_response(state)
    
    context = "\n".join(state["retrieved_docs"])
    query = state["messages"][-1].content
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful AI assistant. Use the following context to answer the user's question accurately and informatively. Context: {context} If the context doesn't contain relevant information, say so politely and provide a general helpful response."),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    chain = prompt | llm | StrOutputParser()
    
    try:
        response = chain.invoke({
            "context": context,
            "messages": state["messages"]
        })
        
        state["messages"].append(AIMessage(content=response))
        state["current_step"] = "completed"
        
    except Exception as e:
        print(f"Error generating RAG response: {e}")
        # Fallback to chat
        return generate_chat_response(state)
    
    return state

print("RAG response function defined!")

In [ ]:
def generate_chat_response(state: ChatState) -> ChatState:
    """Generate conversational response"""
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a friendly and helpful AI assistant powered by Groq. Engage in natural conversation with the user. Be informative, helpful, and engaging."),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    chain = prompt | llm | StrOutputParser()
    
    response = chain.invoke({
        "messages": state["messages"]
    })
    
    state["messages"].append(AIMessage(content=response))
    state["current_step"] = "completed"
    
    return state

print("Chat response function defined!")

In [ ]:
# Create the graph
workflow = StateGraph(ChatState)

# Add nodes
workflow.add_node("should_use_rag", should_use_rag)
workflow.add_node("retrieve_context", retrieve_context)
workflow.add_node("generate_rag_response", generate_rag_response)
workflow.add_node("generate_chat_response", generate_chat_response)

# Add conditional edges from START
workflow.add_conditional_edges(
    START,
    should_use_rag,
    {
        "rag": "retrieve_context",
        "chat": "generate_chat_response"
    }
)

# Add edges for RAG path
workflow.add_edge("retrieve_context", "generate_rag_response")
workflow.add_edge("generate_rag_response", END)

# Add edges for chat path
workflow.add_edge("generate_chat_response", END)

# Compile the graph with memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

print("LangGraph workflow with RAG created!")

In [ ]:
def chat_with_bot(user_input: str, user_id: str = "default_user", config: Dict = None) -> str:
    """Main chat function"""
    if config is None:
        config = {"configurable": {"thread_id": user_id}}
    
    # Create initial state
    state = {
        "messages": [HumanMessage(content=user_input)],
        "user_id": user_id,
        "context": {},
        "retrieved_docs": [],
        "current_step": "start"
    }
    
    # Run the workflow
    result = app.invoke(state, config)
    
    # Return the last AI message
    return result["messages"][-1].content

print("Chat interface ready!")

In [ ]:
# Document Upload Instructions
print("Document Upload Instructions:")
print("=" * 50)
print("To add your own documents to the knowledge base:")
print("1. Place your documents in the same directory as this notebook")
print("2. Supported formats: PDF, TXT, DOCX, MD")
print("3. Use the add_documents_to_knowledge_base() function")
print("4. Example: add_documents_to_knowledge_base(['document.pdf', 'notes.txt'])")
print("5. Use list_knowledge_base_info() to see current knowledge base status")
print("=" * 50)

# Example of how to add documents
print("\nExample usage:")
print("# add_documents_to_knowledge_base(['your_document.pdf'])")
print("# list_knowledge_base_info()")

In [ ]:
# Test the chatbot with RAG
print("Advanced Chatbot Demo with Groq + RAG")
print("=" * 60)

# Example conversations to test both RAG and chat
test_questions = [
    "Hello! How are you today?",  # Chat
    "What is LangGraph?",  # RAG
    "How does RAG work?",  # RAG
    "Tell me about Groq",  # RAG
    "What's the weather like?",  # Chat
    "Can you help me with a coding problem?",  # Chat
    "Explain machine learning",  # RAG
    "What is Python?",  # RAG
    "How are you feeling?",  # Chat
    "Tell me about APIs"  # RAG
]

for question in test_questions:
    print(f"User: {question}")
    try:
        response = chat_with_bot(question)
        print(f"Bot: {response}")
    except Exception as e:
        print(f"Error: {e}")
    print("-" * 60)
    print()

In [9]:
from IPython.display import display, HTML
import ipywidgets as widgets
from IPython.display import clear_output

# Create the chat interface
chat_output = widgets.Output()
input_box = widgets.Text(placeholder='Type your message here...', layout=widgets.Layout(width='80%'))
send_button = widgets.Button(description='Send', button_style='primary')
clear_button = widgets.Button(description='Clear', button_style='warning')

chat_history = []

def on_send_click(b):
    user_message = input_box.value.strip()
    if user_message:
        with chat_output:
            print(f"You: {user_message}")
            
            try:
                # Get bot response
                bot_response = chat_with_bot(user_message)
                print(f"Bot: {bot_response}")
                
                # Store in history
                chat_history.append({"user": user_message, "bot": bot_response})
            except Exception as e:
                print(f"Error: {e}")
            
            print("-" * 50)
        
        input_box.value = ''

def on_clear_click(b):
    chat_output.clear_output()
    chat_history.clear()
    print("Chat history cleared!")

def on_enter_press(change):
    if change['name'] == 'value' and change['new'].endswith('\n'):
        input_box.value = change['new'].rstrip('\n')
        on_send_click(None)

send_button.on_click(on_send_click)
clear_button.on_click(on_clear_click)
input_box.observe(on_enter_press, names='value')

# Display the interface
display(HTML('<h3>Interactive Chat Interface with Groq + RAG</h3>'))
display(HTML('<p>Ask questions about uploaded documents or have a general conversation:</p>'))
display(widgets.HBox([input_box, send_button, clear_button]))
display(chat_output)

print("Interactive chat interface ready! Type your message and press Enter or click Send.")
print("Try asking questions about your uploaded documents or general questions like 'What is LangGraph?'")

Output()

Interactive chat interface ready! Type your message and press Enter or click Send.
Try asking questions about your uploaded documents or general questions like 'What is LangGraph?'
